In [1]:
using Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")

In [2]:
using CSV
using DataFrames

## What is JuMP?

JuMP is a _modeling language_ for optimization problems, writen in Julia. 

When solving an optimization problem, you should use a _solver_ :  a software implementation of an optimization algorithm. Solvers typically need the problem to be written in a very specific format, which can be time-consuming and confusing.

A modeling language (like JuMP) lets you code an optimization problem in a more natural way. It does the translation to the low-level solver format for you.

There are a number of modeling languages out there. Why JuMP?

* User-friendly
* Matches performance of competitors
* Solver-independent
* Easy to extend and take advantage of advanced features

### Installing JuMP

To install JuMP, just run:

In [3]:
#import Pkg
#Pkg.add("JuMP")

We already did this in the preassignment. To actually solve a problem, we will also need to install a solver package. There are 15+ options exposed in julia, each with support for different problem classes, different performance profiles, licensing requirements, etc. For the preassignment, we installed **Gurobi**, a best-of-breed linear/integer programming solver with a generous academic license. A popular open-source alternative is **Cbc**, which can be accessed without a license and allows us to solve integer optimization problems.

In [4]:
#Pkg.add("Gurobi")

### A first example
Let's see how we translate a simple, 2 variable LP to JuMP code.

$$
\begin{align*}
\max_{x,y} \quad& x + 2y \\
\text{s.t.}\quad& x + y \leq 1 \\
& x, y \geq 0.
\end{align*}
$$

First, we load the JuMP and Gurobi libraries.

In [5]:
using JuMP, Gurobi

Next, we construct a model object. This is a container for everything in our optimization problem: variables, constraints, solver options, etc.
Note that when we specify the solver we can also pass additional arguments (depending on the solver we choose). 
If you're experimenting with an optimization model, it's generally a good idea to set a short time limit for the solver calculations. This will (hopefully) force the solver to stop running if you accidentally pass it a very large model that it can't solve. 

In [6]:
model = Model(with_optimizer(Gurobi.Optimizer,TimeLimit=60))

Academic license - for non-commercial use only - expires 2023-08-11


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

Next, we define the two decision variables in our optimization problem. We will use the ``@variable`` macro (a fancy function, essentially). The first argument is the model object to attach the variable to, and the second specifies the variable name and any bounds.

In [7]:
@variable(model, x >= 0)
@variable(model, y >= 0)

y

In [8]:
model

A JuMP Model
Feasibility problem with:
Variables: 2
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, y

We now add the single constraint of our problem using the ``@constraint`` macro. We write it algebraically, exactly as we see it above.

In [9]:
@constraint(model, x + y <= 1)
model

A JuMP Model
Feasibility problem with:
Variables: 2
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, y

We specify the objective function with the ``@objective`` macro.

In [10]:
@objective(model, Max, x + 2y)

x + 2 y

In [11]:
model

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, y

To solve the optimization problem, call the ``optimize!`` function.

In [12]:
optimize!(model)
termination_status(model)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xb93fdaf2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  2.000000000e+00

User-callback calls 27, time in user-callback 0.00 sec


OPTIMAL::TerminationStatusCode = 1

We can now inspect the solution values and optimal cost.

In [13]:
JuMP.value(x)

0.0

In [14]:
JuMP.value(y)

1.0

In [15]:
JuMP.objective_value(model)

2.0

#### Exercise 1

Code and solve the following optimization problem:

$$
\begin{align*}
\min_{x,y} \quad& 3x - y \\
\text{s.t.}\quad& x + 2y \geq 1 \\
& x \geq 0 \\
& 0 \leq y \leq 1.
\end{align*}
$$

In [16]:
## FILL IN HERE ##

#### Exercise 2

Take the problem from exercise 1 and make two copies below.
* In the first copy, add a new constraint to make the problem **infeasible** (i.e., there are no values of x and y that satisfy all the constraints)
* In the second copy, change the constraints or the objective function to make the problem unbounded (i.e., the optimal solution is infinite)

Solve both versions of the problem and look at the ``termination_status`` to see if you have succeeded.

In [17]:
## FILL IN HERE ##

In [18]:
## FILL IN HERE ##

## Constructing the Olympics "Dream Team"
Now, we're ready to start using optimization to formulate and solve real problems. Let's go back to our Olympics example.

First, let's load our 2018-2019 player data. This CSV file contains statistics for active US-born players in the 2018-2019 season. We have also added in information about whether each player appeared in the previous Olympics (2016) and the most recent NBA All-Star game as additional measures. 

Source: https://www.basketball-reference.com/

In [19]:
df = CSV.read("data/NBA_data_2018_2019.csv", DataFrame);
names(df);

Let's see what this data looks like:

In [20]:
first(df,10)

,ID,Player,Username,Pos,Age,Tm,G,MP,PER
,Int64,String31,String15,String7,Int64,String3,Int64,Int64,Float64
1,5,Bam Adebayo,adebaba01,C,21,MIA,82,1913,17.9
2,8,LaMarcus Aldridge,aldrila01,C,33,SAS,81,2687,22.9
3,11,Jarrett Allen,allenja01,C,20,BRK,80,2096,18.5
4,13,Al-Farouq Aminu,aminual01,PF,28,POR,81,2292,13.2
5,20,Carmelo Anthony,anthoca01,PF,34,HOU,10,294,10.9
6,22,Ryan Arcidiacono,arcidry01,PG,24,CHI,81,1961,11.6
7,23,Trevor Ariza,arizatr01,SF,33,TOT,69,2349,12.0
8,24,D.J. Augustin,augusdj01,PG,31,ORL,81,2269,15.7
9,27,Marvin Bagley,baglema01,PF,19,SAC,62,1567,18.9


### A Simple Model
We'll start by coding up the simple model we sketched out in the slides.

In [21]:
model = Model(with_optimizer(Gurobi.Optimizer,TimeLimit=60))

Academic license - for non-commercial use only - expires 2023-08-11


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

### I. Defining the Decision Variables
First we need to create our decision variables. Remember, we want to construct variables:

$$
    x_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ is selected for the team,}\\
                  0 & \text{otherwise.}
                \end{array}
              \right.
$$

These are called **binary** variables since they only take on values between 0 and 1. We need an $x_i$ for all $i=1,\ldots,N$.

In [22]:
N = nrow(df)
@variable(model, x[1:N], Bin)

200-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]
 x[7]
 x[8]
 x[9]
 x[10]
 x[11]
 x[12]
 x[13]
 ⋮
 x[189]
 x[190]
 x[191]
 x[192]
 x[193]
 x[194]
 x[195]
 x[196]
 x[197]
 x[198]
 x[199]
 x[200]

#### II. Formalizing the Objective

We'll start with a basic objective, which is simply to maximize the average Player Efficiency Rating (PER) of the players on the selected team. We'll denote player $i$'s PER as $c_i$. We can then calculate the objective as:

$$ \frac{1}{12} \sum_{i=1}^{N}(x_i*s_i)$$

We can easily formulate this in JuMP:

In [23]:
s = df[!,:PER]

200-element Vector{Float64}:
 17.9
 22.9
 18.5
 13.2
 10.9
 11.6
 12.0
 15.7
 18.9
 12.8
 13.1
 12.3
 11.9
  ⋮
 15.8
 21.7
  7.0
 21.1
 14.8
 23.5
 21.2
 12.6
 12.8
 15.5
 16.2
 17.0

In [24]:
s = df[!,:PER]
@objective(model, Max, 1/12*sum(x[i]*s[i] for i=1:N))

1.4916666666666665 x[1] + 1.9083333333333332 x[2] + 1.5416666666666665 x[3] + 1.0999999999999999 x[4] + 0.9083333333333333 x[5] + 0.9666666666666666 x[6] + x[7] + 1.3083333333333331 x[8] + 1.5749999999999997 x[9] + 1.0666666666666667 x[10] + 1.0916666666666666 x[11] + 1.025 x[12] + 0.9916666666666667 x[13] + 1.7333333333333334 x[14] + 1.1833333333333331 x[15] + 1.075 x[16] + 0.9 x[17] + 0.9833333333333334 x[18] + 1.6083333333333334 x[19] + 1.6833333333333331 x[20] + 0.6333333333333333 x[21] + 0.9 x[22] + 1.0916666666666666 x[23] + 1.4833333333333334 x[24] + 0.575 x[25] + 1.125 x[26] + 1.0666666666666667 x[27] + 1.75 x[28] + 0.875 x[29] + 1.0583333333333331 x[30] + 1.6833333333333331 x[31] + 1.8583333333333334 x[32] + 1.65 x[33] + 1.075 x[34] + 1.0083333333333333 x[35] + 0.9833333333333334 x[36] + 1.5 x[37] + 0.6916666666666667 x[38] + 1.4 x[39] + 1.8166666666666667 x[40] + 1.125 x[41] + 1.3916666666666666 x[42] + 1.7833333333333332 x[43] + 1.1333333333333333 x[44] + 1.0916666666666666 

#### III. Constructing the Constraints 

Now that we have defined our variables and quantified our goal, we need to specify what requirements any team must satisfy. Let's start by just placing a constraint on team size. 

In [25]:
@constraint(model, sum(x[i] for i=1:N) == 12);

#### IV. Solving the Model
We have specified the three key elements of any mixed-integer optimization model: 
- Decision Variables
- Objective 
- Feasibility Constraints

We're ready to solve the model!

In [26]:
model

A JuMP Model
Maximization problem with:
Variables: 200
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.ZeroOne`: 200 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x

In [27]:
optimize!(model);

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1 rows, 200 columns and 200 nonzeros
Model fingerprint: 0x2f1de7f2
Variable types: 0 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 14.9833333
Presolve removed 1 rows and 200 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 25.375 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.537500000000e+01, best bound 2.537500000000e+01, gap 0.0000%

User-callback calls 38, time in user-callback 0.00 sec


Let's see what our results look like: we can look at the values of our decision variables $x_1,\ldots,x_N$ to see which indices were set to 1. These indices correspond to the players that were selected for the team.

In [28]:
selection_indices = value.(x)

200-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [29]:
selected_players = df[selection_indices.==1,:Player]

12-element Vector{String31}:
 "Stephen Curry"
 "Anthony Davis"
 "Andre Drummond"
 "Kevin Durant"
 "Paul George"
 "James Harden"
 "Montrezl Harrell"
 "LeBron James"
 "Kawhi Leonard"
 "Damian Lillard"
 "Karl-Anthony Towns"
 "Hassan Whiteside"

In [30]:
objective_value(model)

25.374999999999996

How well does this match the list of finalists for the Olympics? 

In [31]:
df[selection_indices.==1,[:Player,:Pos,:AllStar,:Olympics2016,:OlympicsFinalist]]

,Player,Pos,AllStar,Olympics2016,OlympicsFinalist
,String31,String7,Int64,Int64,Int64
1,Stephen Curry,PG,1,0,1
2,Anthony Davis,C,1,0,1
3,Andre Drummond,C,0,0,1
4,Kevin Durant,SF,1,1,1
5,Paul George,SF,1,1,1
6,James Harden,PG,1,0,1
7,Montrezl Harrell,C,0,0,1
8,LeBron James,SF,1,0,1
9,Kawhi Leonard,SF,1,0,1


### Making a More Realistic Team

While this team is technically legal under Olympic regulations, it's not enough for us just to look at PER! We want to make sure that we have enough players of each position to fill out the team. We can add a few constraints to make a more useful team. 

In [32]:
## Our original model
model2 = Model(with_optimizer(Gurobi.Optimizer))
@variable(model2, x[1:N], Bin)
@objective(model2, Max, 1/12*sum(x[i]*df[i,:PER] for i=1:N));
@constraint(model2, sum(x[i] for i=1:N) == 12);

Academic license - for non-commercial use only - expires 2023-08-11


#### Position Requirements

Suppose we want at least 4 guards, 4 forwards, and 3 centers. We first define indicators for each player's position. For example, to indicate which players are forwards, we define:


$$
    f_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ is a forward,}\\
                  0 & \text{otherwise.}
                \end{array}
              \right.
$$

We can create similar variables $c_i$ and $g_i$ for center and guard positions. Note that this is **data**; these are not decision variables.

In [33]:
c = ifelse.(df[!,:Pos].=="C",1,0);
f = ifelse.((df[!,:Pos].=="PF").|(df[!,:Pos].=="SF"),1,0);
g = ifelse.((df[!,:Pos].=="PG").|(df[!,:Pos].=="SG"),1,0);

println("Center Count: ", sum(c))
println("Forward Count: ", sum(f))
println("Guard Count: ", sum(g))

Center Count: 33
Forward Count: 67
Guard Count: 96


We can create constraints by summing the position indicator over the players who are selected for the team, like we did with the objective:

$$ \sum_{i=1}^{N}(f_i*x_i) \geq 4$$

We can do the same for the center and guard positions as well.

In [34]:
# Forward constraint
@constraint(model2, sum(f[i]*x[i] for i=1:N) >= 4);

# Center constraint
@constraint(model2, sum(c[i]*x[i] for i=1:N) >= 3);

# Guard constraint
@constraint(model2, sum(g[i]*x[i] for i=1:N) >= 4);

#### Defensive Ability
We want to make sure that our team has good average defensive ability. One (imperfect!) way to measure this is using Defensive Box Plus/Minus (DBPM). We can require that the average DBPM of the chosen team is at least +1. 


$$ \frac{1}{12} \sum_{i=1}^{N}(d_i*x_i) \geq 1$$

where $d_i$ is player $i$'s DBPM. 

In [35]:
d = df[!,:DBPM]
@constraint(model2, 1/12 * sum(d[i]*x[i] for i=1:N) >= 1.0);

#### Experience Constraints
We also want to make sure our team has enough Olympics experience: let's add a constraint that at least 3 selected players were on the 2016 Olympic team.

We use a similar syntax as before:

$$ \sum_{i=1}^{N}(o_i*x_i) \geq 3 $$

where $o_i = 1$ if player $i$ was on the 2016 Olympic team, and 0 otherwise. 


In [36]:
o = df[!,:Olympics2016]
@constraint(model2, sum(o[i]*x[i] for i=1:N) >= 3);

Let's solve our new model.

In [37]:
set_silent(model2)
optimize!(model2)

In [38]:
selected_players2 = sort(df[value.(x).==1,:Player])

12-element Vector{String31}:
 "Andre Drummond"
 "Anthony Davis"
 "Damian Lillard"
 "Hassan Whiteside"
 "James Harden"
 "Jimmy Butler"
 "Karl-Anthony Towns"
 "Kawhi Leonard"
 "Kevin Durant"
 "LeBron James"
 "Paul George"
 "Stephen Curry"

If we compare to the original player list, we see that we have removed Montrezl Harrell (Center) and added Jimmy Butler (Forward).

In [39]:
sort(selected_players)

12-element Vector{String31}:
 "Andre Drummond"
 "Anthony Davis"
 "Damian Lillard"
 "Hassan Whiteside"
 "James Harden"
 "Karl-Anthony Towns"
 "Kawhi Leonard"
 "Kevin Durant"
 "LeBron James"
 "Montrezl Harrell"
 "Paul George"
 "Stephen Curry"

How good is our new model, according to our **objective function**?

In [40]:
objective_value(model2)

25.28333333333333

In [41]:
objective_value(model)

25.374999999999996

This is < 0.1 worse than our original solution: even with these new constraints, we haven't lost much.

### Variations of Our Basic Model


Before we experiment further, we can put our full model into a **function** that will let us easily modify the model for different objectives and datasets.

In [42]:
function create_roster(df, objective_metric)
    m = Model(with_optimizer(Gurobi.Optimizer))
    set_silent(m)
    
    # Define variables
    N = nrow(df)
    @variable(m, x[1:N], Bin)
    
    # Define relevant data columns
    c = ifelse.(df[!,:Pos].=="C",1,0);
    f = ifelse.((df[!,:Pos].=="PF").|(df[!,:Pos].=="SF"),1,0);
    g = ifelse.((df[!,:Pos].=="PG").|(df[!,:Pos].=="SG"),1,0);
    d = df[!,:DBPM];
    o = df[!,:Olympics2016];
    
    # Pull in data for our chosen objective
    s = df[!,objective_metric];
    
    # Objective
    @objective(m, Max, 1/12 * sum(x[i]*s[i] for i=1:N))
    
    # Team size constraint
    @constraint(m, sum(x[i] for i=1:N) == 12);
    
    # Position constraints
    @constraint(m, sum(f[i]*x[i] for i=1:N) >= 4);
    @constraint(m, sum(c[i]*x[i] for i=1:N) >= 3);
    @constraint(m, sum(g[i]*x[i] for i=1:N) >= 4);

    # Experience constraint
    @constraint(m, sum(o[i]*x[i] for i=1:N) >= 3);

    # Defensive ability constraint
    @constraint(m, 1/12 * sum(d[i]*x[i] for i=1:N) >= 1);
    
    optimize!(m)
    
    players = sort(df[value.(x).>0.99,:Player])
    return(objective_value(m), players)
end

create_roster (generic function with 1 method)

Now, it is easy for us to build models for different objective functions. We simply have to specify our dataset of interest and the column of the metric that we want to maximize.

#### Alternative Objective Functions

We chose to optimize average PER, but we could have chosen other metrics too. How does our team composition change when we use other measures of player performance? Try using our function to experiment with different objectives. Remember: you can write `names(df)` to see all of the available columns.

In [43]:
obj_per, players_per = create_roster(df,:PER)

Academic license - for non-commercial use only - expires 2023-08-11


(25.28333333333333, String31["Andre Drummond", "Anthony Davis", "Damian Lillard", "Hassan Whiteside", "James Harden", "Jimmy Butler", "Karl-Anthony Towns", "Kawhi Leonard", "Kevin Durant", "LeBron James", "Paul George", "Stephen Curry"])

In [49]:
obj_bpm, players_bpm = # FILL IN HERE using BPM
obj_ws48, players_ws48 = # FILL IN HERE using WS

Academic license - for non-commercial use only - expires 2023-08-11
Academic license - for non-commercial use only - expires 2023-08-11


(0.2099166666666667, String31["Anthony Davis", "Damian Lillard", "Derrick Favors", "James Harden", "Jimmy Butler", "Kawhi Leonard", "Kevin Durant", "LeBron James", "Mitchell Robinson", "Nerlens Noel", "Paul George", "Stephen Curry"])

#### How different is the playoff perspective?  

Now let's repeat the model (with the original PER objective), but using data from the past two playoff seasons instead. We're looking at average playoff statistics for all active US-born NBA players.

In [46]:
df_playoffs = CSV.read("data/NBA_data_playoffs_2017_2019.csv", DataFrame);
first(df_playoffs,10)

,ID,Player,Username,From,To,Tm,Lg,WS,G
,Int64,String31,String15,Int64,Int64,String3,String3,Float64,Int64
1,1,LeBron James,jamesle01,2017,2018,CLE,NBA,9.4,40
2,2,Kevin Durant,duranke01,2017,2019,GSW,NBA,9.0,48
3,3,Stephen Curry,curryst01,2017,2019,GSW,NBA,8.8,54
4,4,Kawhi Leonard,leonaka01,2017,2019,TOT,NBA,7.7,36
5,5,Draymond Green,greendr01,2017,2019,GSW,NBA,7.2,60
6,6,James Harden,hardeja01,2017,2019,HOU,NBA,5.1,39
7,7,Kyle Lowry,lowryky01,2017,2019,TOR,NBA,4.7,42
8,8,Chris Paul,paulch01,2017,2019,TOT,NBA,4.5,33
9,9,Andre Iguodala,iguodan01,2017,2019,GSW,NBA,3.8,52


We can leverage the same function, but now we want to speficy a new dataset, **df_playoffs**, rather than the original dataset **df**.

In [47]:
obj_playoff, players_playoff = create_roster(df_playoffs,:PER)

Academic license - for non-commercial use only - expires 2023-08-11


(30.866666666666667, String31["Anthony Davis", "Chris Paul", "Draymond Green", "JaVale McGee", "James Harden", "Jimmy Butler", "Kawhi Leonard", "Kevin Durant", "Larry Nance", "LeBron James", "Malcolm Delaney", "Stephen Curry"])

We can create a table to compare how much these different lists overlap. We'll do this using the `join` function, which is similar to SQL syntax.

In [50]:
player_compare = outerjoin(
    DataFrame(Player = players_per, PER = 1), 
    DataFrame(Player = players_bpm, BPM = 1),
    DataFrame(Player = players_ws48, WS_48 = 1), 
    DataFrame(Player = players_playoff, PER_PLAYOFF = 1),
    on = :Player)

sort(player_compare)

,Player,PER,BPM,WS_48,PER_PLAYOFF
,String31,Int64?,Int64?,Int64?,Int64?
1,Andre Drummond,1,missing,missing,missing
2,Anthony Davis,1,1,1,1
3,Chris Paul,missing,missing,missing,1
4,Damian Lillard,1,1,1,missing
5,Derrick Favors,missing,missing,1,missing
6,Draymond Green,missing,missing,missing,1
7,Hassan Whiteside,1,missing,missing,missing
8,JaVale McGee,missing,missing,missing,1
9,James Harden,1,1,1,1


Finally, we can also add a column comparing our lists to the true list of Olympic Finalists for 2020.   

In [51]:
# Pull a dataset of the olympic finalists
olympics_finalists = unique(df[df[!,:OlympicsFinalist].==1,[:Player,:OlympicsFinalist]])

sort(leftjoin(player_compare, olympics_finalists, on = :Player))

,Player,PER,BPM,WS_48,PER_PLAYOFF,OlympicsFinalist
,String31,Int64?,Int64?,Int64?,Int64?,Int64?
1,Andre Drummond,1,missing,missing,missing,1
2,Anthony Davis,1,1,1,1,1
3,Chris Paul,missing,missing,missing,1,1
4,Damian Lillard,1,1,1,missing,1
5,Derrick Favors,missing,missing,1,missing,missing
6,Draymond Green,missing,missing,missing,1,1
7,Hassan Whiteside,1,missing,missing,missing,missing
8,JaVale McGee,missing,missing,missing,1,1
9,James Harden,1,1,1,1,1


7 players (Anthony Davis, James Harden, Jimmy Butler, Kawhi Leonard, Kevin Durant, Lebron James, and Stephen Curry) are chosen in every variation of our model. Certain players only appear when we look at playoff statistics (e.g. Chris Paul, Draymond Green), while others look better when we focus on the regular season (e.g. Paul George, Damian Lillard).